In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import tkinter as tk
from tkinter import simpledialog
import openai
import json
import re
from IPython.display import display

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width', 1000)
openai.api_key = "USE YOUR OpenAI API HERE" 
df = pd.read_csv('data/demo_merged_comment_500.csv')

In [4]:
# raw preview
df.head()


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,combined_text,embedding
0,5.0,Best reward,Love the digital gift cards it's a great way to reward my teens for a job well done. And they can use it immediately.,[],B07C438TMN,B0BL65X86R,AGPILJAJRVYHKAPNJU45FCNZ7LRA,1653678735919,0,True,title: Best reward \ntext: Love the digital gift cards it's a great way to reward my teens for a job well done. And they can use it immediately.,"[-0.0053978245705366135, 0.008798344992101192, -0.013230320066213608, 0.005663888994604349, -0.010817517526447773, 0.010321835987269878, -0.02511938475072384, 0.008608819916844368, -0.001209134585224092, 0.03600979596376419, 0.01456428598612547, -0.031052982434630394, -0.026621008291840553, -0.014724653214216232, -0.0039034911897033453, -0.0007790580857545137, 0.005160917993634939, 0.020220885053277016, -0.018981682136654854, 0.008448452688753605, -0.019754361361265182, -0.0017449079314246774, 0.004811025224626064, 0.009534577839076519, 0.01851515844464302, -0.011816170066595078, -0.021051881834864616, -0.02456538751721382, -0.014491391368210316, 0.014017579145729542, 0.004566829651594162, 0.027495739981532097, -0.04563184827566147, -0.026067011058330536, 0.0028210103046149015, -0.020629093050956726, -0.01567227952182293, -0.02672305889427662, -0.0037941497284919024, -0.004176844842731953, -0.00244013755582273, 0.003939938265830278, -0.011633934453129768, -0.018354790285229683, -0...."
1,1.0,NOT MINE I DID NOT ORDER THIS NOT HAPPY ABOUT IT,Not mine another person used my account to get this I'm not happy at all it's happened twice now,[],B07C438TMN,B0BL65X86R,AEZLFV6AMF7RCWSXQE36FRBQ4RCQ,1571074149337,0,True,title: NOT MINE I DID NOT ORDER THIS NOT HAPPY ABOUT IT \ntext: Not mine another person used my account to get this I'm not happy at all it's happened twice now,"[-0.017139893025159836, 0.0024524058680981398, 0.0020674129482358694, -0.03951568901538849, -0.025640539824962616, 0.005855744704604149, -0.029136277735233307, -0.0029240224976092577, -0.02873588353395462, 0.04185644909739494, 0.04542918503284454, -0.06621880829334259, -0.024701155722141266, -0.03394098952412605, 0.03381779417395592, 0.020805027335882187, -0.06307727098464966, 0.028674285858869553, -0.0008330288110300899, -0.013189863413572311, 0.0024139066226780415, 0.00944003090262413, 0.0368669368326664, 0.026656921952962875, 0.03825291246175766, -0.007072323001921177, -0.022914789617061615, -0.006152189802378416, 0.012158081866800785, -0.041271258145570755, -0.02245279774069786, 0.01427554339170456, -0.0022541345097124577, 0.018418069928884506, 0.05084988847374916, 0.013074365444481373, -0.017786681652069092, -0.011033901944756508, 0.0041579254902899265, 0.03868410736322403, 0.0012146532535552979, -0.021297818049788475, -0.003397563938051462, 0.0026872516609728336, -0.012543074..."
2,1.0,i didn't get the code,i bought the card with my money but cant see the card number i am not sure why but it'll be nice if i get the code for my card.,[],B07C438TMN,B0BL65X86R,AG7ZKV5KORR2ZZPE43YRPDOAQZBQ,1614124036228,0,True,title: i didn't get the code \ntext: i bought the card with my money but cant see the card number i am not sure why but it'll be nice if i get the code for my card.,"[0.002988348715007305, 0.05137544870376587, 0.0029336377047002316, -0.014956836588680744, -0.012957056984305382, -0.021491961553692818, -0.010723342187702656, 0.02872135117650032, -0.02251826412975788, 0.051616933196783066, -0.011515707708895206, -0.017643330618739128, -0.019077135249972343, -0.012013765051960945, 0.02211076207458973, 0.023001229390501976, 0.005395629443228245, 0.032479427754879, -0.023529473692178726, -0.018518706783652306, -0.013379652053117752, 0.004629676695913076, 0.03281146660447121, 0.03779204562306404, 0.0009503665496595204, -0.011945848353207111, -0.05046989023685455, -0.0018384759314358234, 0.02337854541838169, -0.037641119211912155, 0.0260650422424078, 0.008987684734165668, -0.028962833806872368, 0.019

In [5]:
# Cleaned preview
long_unrelated_df = df[df['text'].str.len() > 500]
display(long_unrelated_df[['rating', 'title', 'text', 'helpful_vote']])

,rating,title,text,helpful_vote
31,1.0,$100 Gift Cards Are Broken. Cant Complete Transaction or Cancel it After.,"So I have tried to purchases a $100 Gift Card 2 different times in the last few Weeks from Amazon Thinking the 1st time was a fluke maybe But yet again I got the same Problem I get a Message Saying. (Payment revision needed Please update your payment method) Fallowed By and Email that States(Due to a problem with the payment method you provided, we couldn’t charge your account for your order (#xxx).We may attempt to reprocess payment for your order in a few days using the payment method you provided.Please go to the following page to make sure that the payment information for your account is correct, and so that we have a valid payment method for your future orders:If we don't receive payment from you within the next 7 day(s), your order will be cancelled.)The Problem With all of this is that there is nothing wrong with my payment setup and it works fine for literally everything else Including other PlayStation Gift Cards of Different Amounts. The other problem I have is there is ...",0
114,5.0,Simple. Straightforward. Fast.,"Honestly, you should ignore all of the negative reviews. Those negative reviews are coming from customers who don’t know how to read and follow simple instructions. It literally doesn’t get any easier. You choose which type of gift card (in this case, Playstation). Then you choose the amount (ranging from $10, $25, $50, or $100). You pay for the digital gift card. Then you check your email for a confirmation of your order. Within that very email, you should receive the redeem code for the gift card. It literally doesn’t get any easier than that. In the case you do have a problem with the redeem code, simply contact Amazon customer service and they will guide you through a new one.",10
177,5.0,Save a Penny!,"Ok, the title says it all. You can save a penny every time you buy online. May be trivial to some, but every penny does count. I'm also in the same boat like some other reviewers. The PSN store stopped accepting my credit card for whatever reason. It was impossible to get it to work. I didn't want to wait a week for it to arrive in the mail, but amazon started doing this years ago and THANK YOU!It comes in about 2 min into your email account. I'm also overseas working, so this was a real life saver. Now if Sony would only fix their system to accept my card! But I guess it's better to do it this way and have a budget on what I'm buying. Anyway, great service and great price.",0
207,1.0,Think twice before buying,"Took several calls to support to obtain digital code. First call, smaller told us to YouTube how to access our code, then provided us with a Google link to a search with misspelled words and eventually told us that we needed to install Amazon on our PS4 and after loging I. From the PS4, our digital code would automatically work. The support person said he was with Sony and not Amazon, however they emailed from an amazing email account. Took several calls and several days to obtain the digital code. I do not recommend purchasing from here.",0
212,5.0,$10 PLAYSTATION STORE DIGITAL CODE GIFT CARD - Fast & Easy To Use!,"Every now and then my son will sucker me into buying him one or two of these $10 Playstation Store Digital Code Gift Cards. Apparently I have purchased them a total of 51 times now... Wow! Unfortunately I can never find this small of an amount at any of the local stores who only seem to carry amounts of $25 or more. Within a minute or two of ordering this gift card on amazon I'm sent an email with the code, my son enters it on his Playstation account and we're done. I like that it's all digital too and that there is no waste for the trash can. I've not had a problem yet receiving any of the codes and if you don't see it in your email you can always find it under the ""Digital content and devices"" section and then clicking on ""Digital games and software.""",9
228,5.0,"Pr

In [7]:
def get_user_query(query_prompt):
    """ Function to get user input and rephrase it with LLM """
    original_query = input("Hi, customer, what is your question?\nOriginal query: ")
    # print(f"You entered: {query}")

    combined_text = query_prompt + original_query

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": combined_text}
        ]
    )

    return response['choices'][0]['message']['content'], original_query

def generate_query_embedding(query):
    """ Function to generate query embedding using OpenAI's embedding model """
    response = openai.Embedding.create(
        model="text-embedding-3-large",
        input=query
    )
    # print(response)
    return response['data'][0]['embedding']

def calculate_similarity(query_embedding, df_embeddings):
    """ Calculate cosine similarity between query embedding and each comment embedding """
    query_embedding = np.array(query_embedding).reshape(1, -1)

    df_embeddings = [np.array(json.loads(embedding)) if isinstance(embedding, str) else np.array(embedding) for embedding in df_embeddings]
    similarities = cosine_similarity(query_embedding, np.vstack(df_embeddings))
    return similarities.flatten()

def filter_and_sort_comments(df, similarities, threshold=0, top_n=4):
    """ Filter and sort comments based on similarity """
    df['similarity'] = similarities
    df_filtered = df[df['similarity'] > threshold]
    df_sorted = df_filtered.sort_values(by='similarity', ascending=False).reset_index(drop=True)
    # df_sorted = df_filtered.sort_values(by='similarity', ascending=False)

    # print(df_sorted[['rating',  'text',  'similarity']])
    return df_sorted.head(top_n)

In [11]:
# 1. Get user query
query_prompt='''
    You will be given a question about a product sold on Amazon, which will then be converted into an embedding by GPT-4.
    Additionally, I have a database of Amazon product reviews. Remember: Your answer will be directly embedded with GPT, and
    then it will be compared with the comment embeddings in the database to find the most relevant comments. Therefore, do not
    include any unnecessary words or phrases in your response, and rephrase the question in a concise manner. Most importantly,
    categorize the question into specific product attributes such as 'price', 'quality', 'usability', 'convenience', etc., based on
    the user's query.

    Examples:
    User: I heard this card sucks, is it true?
    Assistant: Quality of the card

    User: Is this toy extremely expensive?
    Assistant: Price of the toy

    User: Is this tool suitable for man?
    Assistant: Target user of the tool

    User: Can I use this card outside of the US?
    Assistant: Usability of the card (region restriction)

    User: How quickly will I receive the code?
    Assistant: Convenience (delivery time)

    Now comes the question:
      '''

query, original_query = get_user_query(query_prompt) # for example, we can use the delivery speed of the card here
# print('\nRephrased question by gpt-4o:', query)

# 2. Generate query embedding
query_embedding = generate_query_embedding(original_query)

# 4. Calculate similarities
similarities = calculate_similarity(query_embedding, df['embedding'])

# 5. Filter and sort comments
num_return = 6  # Number of comments returned for this demo.
# We use 6 comments here instead of the 8 or 10 used in the paper, 
# as a quick example to reduce runtime and token consumption when calling the LLM.


filtered_comments = filter_and_sort_comments(df, similarities, top_n=num_return)
print('\nSource comments:')
display(filtered_comments[['rating', 'title', 'text', 'similarity', 'helpful_vote']])


Source comments:


,rating,title,text,similarity,helpful_vote
0,5.0,Fast delivery,The E-Card came through right away. Would def recommend for last minute gift needed in a hurry!,0.451621,0
1,5.0,Digital cards are the way to go!,I like how this order is delivered digitally and very quickly.,0.441105,0
2,5.0,Was playing the moment my card went through.,Quickest was to get a game card I found. Literally a minute and a half.,0.435196,0
3,5.0,Fast digital speed,Can’t say much but appreciate how fast the speed was in receiving my code. I often do give aways and man it helps a ton!,0.421779,0
4,5.0,Quick fast way to secure a points card,Great buy and quick response time to receive the digital card.,0.403743,0
5,5.0,Fast delivery,"Well, nothing much to say when it's a digital code gift card, but it was fast (almost received the code instantly), redeemed and the process was pretty much simple.",0.403713,0


In [21]:
# demo run
import importlib
from src.llm_evaluation import evaluate_with_gpt
from src.llm_summarization import (
    label_comment_relevance,
    generate_final_output
)
from src.shapley_calculation import calculate_shapley_values


shapley_values = calculate_shapley_values(filtered_comments, original_query, generate_final_output, evaluate_with_gpt, num_evaluations=4)

summary_0_output:
The delivery speed of the E-Card is notably fast, as it was received immediately, making it a recommended option for those in need of a last-minute gift [0].

summary_1_output:
The comment highlights the advantage of digital cards, emphasizing their quick delivery speed, which is appreciated by the user [0].

summary_2_output:
The user reports that the delivery speed of the card was extremely fast, taking only a minute and a half to receive the game card after the transaction was completed [0].

summary_3_output:
The user appreciates the fast delivery speed of receiving their code, which is beneficial for conducting giveaways efficiently [0].

summary_4_output:
The comment highlights the quick response time for receiving the digital card, indicating a fast delivery speed for the points card purchase [0].

summary_5_output:
The delivery speed of the digital code gift card was very fast, with the code being received almost instantly, making the redemption process simple